# Intro

## Load

### Standards

In [1]:
import os, sys, pickle
import numpy as np
from tqdm.notebook import trange, tqdm
import re

In [2]:
import matplotlib.pyplot as plt
plt.rcParams['axes.linewidth'] = 2
plt.rcParams['xtick.major.size'] = 10
plt.rcParams['xtick.major.width'] = 2
plt.rcParams['ytick.major.size'] = 10
plt.rcParams['ytick.major.width'] = 2

plt.rcParams['xtick.labelsize'] = 14
plt.rcParams['ytick.labelsize'] = 14

plt.rcParams['xtick.minor.size'] = 5
plt.rcParams['xtick.minor.width'] = 1
plt.rcParams['ytick.minor.size'] = 5
plt.rcParams['ytick.minor.width'] = 1


In [3]:
from multiprocess import Pool

### Homemade modules

In [4]:
from tesste import tesste
from ppp import ppp
from readers_and_converters import file2text, text2dict, text2vec

## Stoopid infos

In [5]:
L_SDGS=17

# Data

In [6]:
with open ('./data/energy_utilities_texts.pickle', 'rb') as f:
    text_dict=pickle.load(f)

# Semantic analysis per report

## Cosine similarity calculations

In [7]:
cos_sim_dict={}
for year in range(2015, 2024):
    s_year=str(year)
    cos_sim_dict[year]={}
    energy_co_s=list(text_dict['Energy'][s_year].keys())
    # inside energy
    energy_cos_sims=np.zeros((len(energy_co_s), len(energy_co_s)))
    energy_pvals=np.zeros((len(energy_co_s), len(energy_co_s)))
    for i_e, energy_co_0 in enumerate(tqdm(energy_co_s, leave=False, desc='Energy '+s_year)):
        text_0=text_dict['Energy'][s_year][energy_co_0]
        for j_e in range(i_e+1, len(energy_co_s)):
            energy_co_1=energy_co_s[j_e]
            text_1=text_dict['Energy'][s_year][energy_co_1]
            cacca=tesste(text_0, text_1)
            energy_cos_sims[i_e, j_e]=cacca.cos_sim
            energy_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Energy']={}
    cos_sim_dict[year]['Energy']['cos_sim']=energy_cos_sims
    cos_sim_dict[year]['Energy']['pvals']=energy_pvals
    
    utilities_co_s=list(text_dict['Utilities'][s_year].keys())
    utilities_cos_sims=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    utilities_pvals=np.zeros((len(utilities_co_s), len(utilities_co_s)))
    # inside utilities
    for i_e, utilities_co_0 in enumerate(tqdm(utilities_co_s, leave=False, desc='Utilities '+s_year)):
        text_0=text_dict['Utilities'][s_year][utilities_co_0]
        for j_e in range(i_e+1, len(utilities_co_s)):
            utilities_co_1=utilities_co_s[j_e]
            text_1=text_dict['Utilities'][s_year][utilities_co_1]
            cacca=tesste(text_0, text_1)
            utilities_cos_sims[i_e, j_e]=cacca.cos_sim
            utilities_pvals[i_e, j_e]=cacca.pval
            
    cos_sim_dict[year]['Utilities']={}
    cos_sim_dict[year]['Utilities']['cos_sim']=utilities_cos_sims
    cos_sim_dict[year]['Utilities']['pvals']=utilities_pvals


Energy 2015:   0%|          | 0/2 [00:00<?, ?it/s]

/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:118: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10020). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at ../c10/cuda/CUDAFunctions.cpp:108.)
  return torch._C._cuda_getDeviceCount() > 0
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
/home/sarawalk/sdgs_py38/lib/python3.8/site-packages/torch/cuda/__init__.py:619: UserWarning: Can't initialize NVML
  warnings

  0%|          | 0/1000 [00:00<?, ?it/s]


KeyboardInterrupt



### Something I forgot

In [ ]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'rb') as f:
    cos_sim_dict=pickle.load(f)

In [ ]:
for year in range(2015, 2024):
    s_year=str(year)
    for field in ['Energy', 'Utilities']:
        firms=list(text_dict[field][s_year].keys())
        cos_sim_dict[year][field]['companies']=firms

In [ ]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'wb') as f:
    pickle.dump(cos_sim_dict, f)

## Cosine similarity analysis

In [10]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'rb') as f:
    cos_sim_dict=pickle.load(f)

In [11]:
years=list(range(2015, 2024))

In [12]:
# actually, it shouldn't be done, but I saw that relaxing from alpha=0.01 to alpha=0.05,
# I get more validated entries...
alpha=.05

In [13]:
for year in years:
    for field in ['Energy', 'Utilities']:
        pvals=cos_sim_dict[year][field]['pvals']
        cos_sim=cos_sim_dict[year][field]['cos_sim']
        firms=cos_sim_dict[year][field]['companies']
        if len(firms)>1:
            # get the number of tests
            n_tests=len(firms)*(len(firms)-1)/2
            # calculate effective alpha according to Bonferroni
            alpha_bonf=alpha/n_tests
            # get the effective alpha according to fdr
            pvals_vec=np.sort(pvals[pvals>0])
            fdr_vec=(1+np.arange(len(pvals_vec)))*alpha_bonf
            alpha_fdrs=fdr_vec[pvals_vec<=fdr_vec]
            if len(alpha_fdrs)>0:
                alpha_fdr=alpha_fdrs[-1]
                # where are the validated entries?
                where_vals=np.where(np.logical_and(pvals<=alpha_fdr, pvals>0))
                val_couples=[]
                for i in range(len(where_vals[0])):
                    val_couples.append((firms[where_vals[0][i]],firms[where_vals[1][i]]))
                # return values
                print(year, field)
                print('---------------------')
                for i, val_couple in enumerate(val_couples):
                    print(val_couple, cos_sim[where_vals[0][i], where_vals[1][i]])

2019 Utilities
---------------------
('02_SSE_PLC', '05_CENTRICA_PLC') 0.8133289217948914
('02_SSE_PLC', '08_RENEWI_PLC') 0.8001610636711121
('05_CENTRICA_PLC', '08_RENEWI_PLC') 0.7937230467796326
('05_CENTRICA_PLC', '10_GOOD_ENERGY_GROUP_PLC') 0.8450196981430054


In [12]:
cos_sim_dict[2019]['Utilities']['companies']

['02_SSE_PLC', '05_CENTRICA_PLC', '08_RENEWI_PLC', '10_GOOD_ENERGY_GROUP_PLC']

Quite bizarre that only on a single year (btw, before the COVID, i.e. before when I would have expected to see some excess in the cosine similarity) there is such a diffuse similarity. Moreover, CENTRICA is (ehm...) the most central, since it is significantly similar to all others.

In [15]:
validated_cos_sim=(2019, 'Utilities', val_couples)

### SDGs covered and similarities

In [14]:
sdgs_dict={}
for year in trange(2015, 2024):
    s_year=str(year)
    sdgs_dict[year]={}
    for field in ['Energy', 'Utilities']:
        sdgs_dict[year][field]={}
        for key in text_dict[field][s_year].keys():
            text=text_dict[field][s_year][key]
            sdgs_dict[year][field][key]=text2dict(text)

  0%|          | 0/9 [00:00<?, ?it/s]

In [32]:
with open ('./data/sdgs_dict.pickle', 'wb') as f:
    pickle.dump(sdgs_dict, f)

### SDGs superposition

In [7]:
with open ('./data/energy_utilities_cos_sim_mats.pickle', 'rb') as f:
    cos_sim_dict=pickle.load(f)

In [8]:
with open ('./data/sdgs_dict.pickle', 'rb') as f:
    sdgs_dict=pickle.load(f)

In [16]:
v_year=validated_cos_sim[0]
v_field=validated_cos_sim[1]

In [17]:
companies=list(sdgs_dict[v_year][v_field].keys())

In [18]:
sdg_superpos=np.zeros((len(companies),len(companies)))
for i_c, company_0 in enumerate(companies):
    sdgs_i_c=list(sdgs_dict[v_year][v_field][company_0].keys())
    for j_c, company_1 in enumerate(companies):
        if i_c!=j_c:
            sdgs_j_c=list(sdgs_dict[v_year][v_field][company_1].keys())
            sdg_superpos[i_c, j_c]=len([sdg for sdg in sdgs_i_c if sdg in sdgs_j_c])/len(sdgs_i_c)

In [19]:
sdg_superpos

array([[0.        , 0.66666667, 0.46666667, 0.73333333],
       [1.        , 0.        , 0.4       , 0.8       ],
       [0.875     , 0.5       , 0.        , 0.75      ],
       [1.        , 0.72727273, 0.54545455, 0.        ]])

Not such a huge similarity. Where does the similarity come from? **To-Do**: manually read the reports.

# Semantic analysis per SDG

In [76]:
with open ('./data/sdgs_dict.pickle', 'rb') as f:
    sdgs_dict=pickle.load(f)

In [77]:
years=list(range(2015, 2024))

In [78]:
fields=['Energy', 'Utilities']

## Check

The conversion to a dictionary was performed thinking about the structure of a sample of the summaries by ChatGPT. Nevertheless, there are some weird entries, as the 2017 report of '08_RENEWI_PLC'. Let's collect them:

In [87]:
sdgs_number=[]
sdgs=[]
for year in tqdm(years):
    #if len(sdgs)<17:
    for field in fields:
        firms=sdgs_dict[year][field].keys()
        for firm in firms:
            _sdgs=list(sdgs_dict[year][field][firm].keys())
            counter=0
            #while len(sdgs)<17 and counter<len(_sdgs):
            while counter<len(_sdgs):
                sdg=_sdgs[counter]
                sdg=sdg.split(':')[0]
                if sdg not in sdgs_number:
                    sdgs_number.append(sdg)
                    sdgs.append(_sdgs[counter])
                elif _sdgs[counter] not in sdgs:
                    print(year, field, firm, sdg)
                counter+=1
sdgs.sort()

  0%|          | 0/9 [00:00<?, ?it/s]

2017 Utilities 08_RENEWI_PLC SDG 5
2017 Utilities 08_RENEWI_PLC SDG 7
2017 Utilities 08_RENEWI_PLC SDG 9
2017 Utilities 08_RENEWI_PLC SDG 11
2017 Utilities 08_RENEWI_PLC SDG 13
2017 Utilities 08_RENEWI_PLC SDG 15
2017 Utilities 08_RENEWI_PLC SDG 17
2023 Utilities 03_SEVERN_TRENT_PLC SDG 3
2023 Utilities 03_SEVERN_TRENT_PLC SDG 5
2023 Utilities 03_SEVERN_TRENT_PLC SDG 7
2023 Utilities 03_SEVERN_TRENT_PLC SDG 9
2023 Utilities 03_SEVERN_TRENT_PLC SDG 11
2023 Utilities 03_SEVERN_TRENT_PLC SDG 15
2023 Utilities 03_SEVERN_TRENT_PLC SDG 17


In [89]:
sdgs.sort(key= lambda x: int(x.split(':')[0].split(' ')[1]))

In [92]:
sdgs

['SDG 1: No Poverty',
 'SDG 2: Zero Hunger',
 'SDG 3: Good Health and Well-being',
 'SDG 4: Quality Education',
 'SDG 5: Gender Equality',
 'SDG 6: Clean Water and Sanitation',
 'SDG 7: Affordable and Clean Energy',
 'SDG 8: Decent Work and Economic Growth',
 'SDG 9: Industry, Innovation, and Infrastructure',
 'SDG 10: Reduced Inequality',
 'SDG 11: Sustainable Cities and Communities',
 'SDG 12: Responsible Consumption and Production',
 'SDG 13: Climate Action',
 'SDG 14: Life Below Water',
 'SDG 15: Life on Land',
 'SDG 16: Peace, Justice, and Strong Institutions',
 'SDG 17: Partnerships for the Goals']

In [91]:
with open('./data/sdgs.pickle', 'wb') as f:
    pickle.dump(sdgs, f)

## Rethinking the various reading functions

In [98]:
bad_ones=[text_dict['Utilities']['2017']['08_RENEWI_PLC'], text_dict['Utilities']['2023']['03_SEVERN_TRENT_PLC']]
standard_one=text_dict['Energy']['2020']['01_shell']

In [102]:
bad_ones[0]

"\nSummary of LSE_RWI_2017.pdf in Relation to United Nations' Sustainable Development Goals (SDGs)\nSDG 1: No Poverty The document emphasizes efforts to ensure affordability of water services for customers, particularly those in vulnerable situations. Initiatives like the WaterCare+ program provide financial support and benefit entitlement checks to help customers manage their bills and avoid water poverty.\nSDG 2: Zero Hunger Not explicitly mentioned.\nSDG 3: Good Health and Well-being The report highlights the importance of clean and safe water supply as essential for public health. Various programs ensure water quality and safety, including maintaining high standards for drinking water and managing wastewater effectively to prevent public health risks.\nSDG 4: Quality Education Not explicitly mentioned.\nSDG 5: Gender Equality The document mentions efforts to promote diversity and inclusion within the company, including initiatives to improve gender equality among employees.\nSDG 6:

In [103]:
text2dict(bad_ones[0])

{'SDG 5: Gender Equality The document mentions efforts to promote diversity and inclusion within the company, including initiatives to improve gender equality among employees.': 'SDG 6: Clean Water and Sanitation A significant focus is placed on ensuring a reliable supply of clean water and efficient wastewater management. Investments in infrastructure and technology aim to maintain high standards of water quality and sanitation.',
 'SDG 7: Affordable and Clean Energy Not explicitly mentioned.': 'SDG 8: Decent Work and Economic Growth The company emphasizes the creation of jobs and economic opportunities, particularly through investments in infrastructure projects and community engagement. Initiatives like the WaterShare+ scheme aim to provide economic benefits to customers and communities.',
 'SDG 9: Industry, Innovation, and Infrastructure Investments in advanced water treatment facilities and innovative solutions for water management are highlighted. The company focuses on upgrading

In [105]:
text=standard_one

In [196]:
def new_text2dict(text):
    # the information contained here is the same of the entire text, but organised per SDG...
    sdgs_dict={}
    counter=0
    aux=text.split('\n')
    while counter<len(aux):
        _a=aux[counter]
        try:
            if len(_a)>0:
                if 'SDG' in _a:
                    sdg=[sdg for sdg in sdgs if sdg in _a]
                    if len(sdg)==1:
                        sdg=sdg[0]
                        # well-behaving text
                        if _a==sdg and 'Not explicitly mentioned' not in aux[counter+1]:
                            sdgs_dict[_a]=aux[counter+1]
                            counter+=2
                        elif _a==sdg and 'Not explicitly mentioned' in aux[counter+1]:
                            counter+=2
                        else:
                            # not well-behaving text
                            _text=_a.split(sdg)[1]
                            if 'Not explicitly mentioned' not in _text:
                                sdgs_dict[sdg]=_text.strip().replace('\u200b', '')
                            counter+=1
                    else:
                        #print(_a)
                        #raise Error
                        counter+=1
                else:
                    counter+=1
            else:
                counter+=1
        except:
            print(_a)
            raise Exception('Mannaggia cavolini!')
    return sdgs_dict

In [197]:
for key in tqdm(text_dict.keys()):
    for s_key in text_dict[key].keys():
        for s_s_key in text_dict[key][s_key].keys():
            if (key, s_key, s_s_key) not in [('Utilities','2017','08_RENEWI_PLC'), ('Utilities','2023','03_SEVERN_TRENT_PLC')]:
                text=text_dict[key][s_key][s_s_key]
                old_dict=text2dict(text)
                new_dict=new_text2dict(text)
                if old_dict!=new_dict:
                    # on correctly formatted tests the routine works correctly
                    print(key, s_key, s_s_key)
                    #for final_key in old_dict.keys():
                    #    print(final_key)
                    #    print(old_dict[final_key])
                    #    print(new_dict[final_key])
                    

  0%|          | 0/2 [00:00<?, ?it/s]

In [198]:
bad_ones[1]

"\nSummary of SSE Sustainability Report 2023 in the Context of United Nations' Sustainable Development Goals (SDGs)\nSDG 1: No Poverty Not explicitly mentioned.\nSDG 2: Zero Hunger Not explicitly mentioned.\nSDG 3: Good Health and Well-being SSE focuses on the health and safety of its employees and communities. The report mentions initiatives aimed at ensuring employee well-being, mental health support, and maintaining a safe working environment. SSE also supports good health through water and sanitation projects in collaboration with WaterAid\u200b\u200b\u200b\u200b.\nSDG 4: Quality Education SSE invests in educational programs, including apprenticeship schemes and graduate programs. The company emphasizes lifelong learning and development opportunities for its employees and community members\u200b\u200b.\nSDG 5: Gender Equality SSE is committed to promoting gender equality within its workforce. The company aims for gender parity in leadership positions and ensures fair pay practices.

In [200]:
new_text2dict(bad_ones[0])

{'SDG 1: No Poverty': 'The document emphasizes efforts to ensure affordability of water services for customers, particularly those in vulnerable situations. Initiatives like the WaterCare+ program provide financial support and benefit entitlement checks to help customers manage their bills and avoid water poverty.',
 'SDG 3: Good Health and Well-being': 'The report highlights the importance of clean and safe water supply as essential for public health. Various programs ensure water quality and safety, including maintaining high standards for drinking water and managing wastewater effectively to prevent public health risks.',
 'SDG 5: Gender Equality': 'The document mentions efforts to promote diversity and inclusion within the company, including initiatives to improve gender equality among employees.',
 'SDG 6: Clean Water and Sanitation': 'A significant focus is placed on ensuring a reliable supply of clean water and efficient wastewater management. Investments in infrastructure and t

In [201]:
sdg_dict=new_text2dict(bad_ones[0])

In [207]:
def text2vec(sdgs_dict):
    # Here I brutally binarized by presence/absence of mention in the report of the given SDG
    # there are 17 sdgs, starting from the entries from text2dict
    sdgs_vec=np.zeros(17, dtype=int)
    for key in sdg_dict.keys():
        which=int(key.split(':')[0].split(' ')[1])-1
        sdgs_vec[which]=1
    return sdgs_vec

In [208]:
text2vec(sdgs_dict)

array([1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [209]:
sdg_dict.keys()

dict_keys(['SDG 1: No Poverty', 'SDG 3: Good Health and Well-being', 'SDG 5: Gender Equality', 'SDG 6: Clean Water and Sanitation', 'SDG 8: Decent Work and Economic Growth', 'SDG 9: Industry, Innovation, and Infrastructure', 'SDG 10: Reduced Inequality', 'SDG 11: Sustainable Cities and Communities', 'SDG 12: Responsible Consumption and Production', 'SDG 13: Climate Action', 'SDG 14: Life Below Water', 'SDG 15: Life on Land', 'SDG 16: Peace, Justice, and Strong Institutions', 'SDG 17: Partnerships for the Goals'])

In [206]:
for key in sdg_dict.keys():
    which=int(key.split(':')[0].split(' ')[1])-1
    print(which)

0
2
4
5
7
8
9
10
11
12
13
14
15
16


## Debug

In [212]:
    with open ('./data/sdgs_dict.pickle', 'rb') as f:
        sdgs_dict=pickle.load(f)
    

In [213]:
sdgs_dict

{2015: {'Energy': {'01_shell': {'SDG 3: Good Health and Well-being': 'Shell’s 2015 report emphasizes its commitment to safety and health. It mentions improvements in safety performance, although it also notes incidents, including fatalities, that deeply affected the company. Shell continues to focus on improving safety standards and reducing health risks for its employees and communities.',
    'SDG 4: Quality Education': "The report highlights Shell's efforts in community engagement, particularly in Nigeria, where the company invests in educational initiatives. This includes funding for schools and scholarships to improve access to quality education in local communities.",
    'SDG 5: Gender Equality': 'Shell’s report discusses efforts to promote diversity and inclusion within the company. It focuses on increasing female representation in the workforce and leadership positions, and it outlines policies to support gender equality and empower women.',
    'SDG 6: Clean Water and Sanitat

In [215]:
test=True

In [216]:
    keys=list(sdgs_dict.keys())
    
    if test:
        keys=keys[:2]
    

In [217]:
keys

[2015, 2016]

In [220]:
def cos_sim_per_sdg(dict_0, dict_1, tqdm_loops=False, verbose=False, n_sample=10**3):
    common_sdgs=[key for key in dict_0.keys() if key in dict_1.keys()]
    for sdg in common_sdgs:
        aux=tesste(dict_0[sdg], dict_1[sdg], tqdm_loops=tqdm_loops, n_sample=n_sample, verbose=False)
        
    return aux

In [225]:
    
    csps_dict={}
    for key in tqdm(keys):
        # the first key is the year
        csps_dict[key]={}
        for sector in ['Energy', 'Utilities']:
            firms=list(sdgs_dict[key][sector].keys())
            csps_dict[key][sector]={}
            csps_dict[key][sector]['cos_sim']=np.zeros((len(firms), len(firms)))
            csps_dict[key][sector]['theo_pvals']=np.zeros((len(firms), len(firms)))
            csps_dict[key][sector]['sam_pvals']=np.zeros((len(firms), len(firms)))
                                                        
            # the third key is the firm
            for i in range(len(firms)):
                firm_i=firms[i]
                dict_i=sdgs_dict[key][sector][firm_i]
                for j in range(i+1, len(firms)):
                    firm_j=firms[j]
                    dict_j=sdgs_dict[key][sector][firm_j]
                    csps=cos_sim_per_sdg(dict_i, dict_j, n_sample=20,tqdm_loops=True)
                    csps_dict[key][sector]['cos_sim'][i, j]=csps.cos_sim
                    csps_dict[key][sector]['theo_pvals'][i, j]=csps.pval
                    csps_dict[key][sector]['sam_pvals'][i, j]=csps.sampled_pval
   

  0%|          | 0/2 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

...

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

  0%|          | 0/20 [00:00<?, ?it/s]

In [228]:
                 
    with open('./data/csps_dict_test.pickle', 'wb') as f:
        pickle.dump(csps_dict, f)

In [229]:
with open('./data/csps_dict_test.pickle', 'rb') as f:
        csps_dict_check=pickle.load(f)

In [237]:
if bool('True'):
    print('ok')

ok
